#### Pandas final assignment

Explore the hypothesis that given that an area has two sports teams in different sports, those teams will perform the 
same within their respective sports. Use paired t-tests between all pairs of sports. Use 2018 data only.
 

In [65]:
import re
import numpy as np
import pandas as pd
import scipy.stats as stats

In [66]:
def cities_master(sport):
    
    cities = pd.read_html("datafiles/wikipedia_data.html")[1]
    cities = cities.iloc[0:-1]
    cities = cities[['Metropolitan area','Population (2016 est.)[8]',"NHL","MLB","NBA","NFL"]]
    cities.rename(columns = {"Population (2016 est.)[8]":"pop"}, inplace = True)
    cities["pop"]= pd.to_numeric(cities["pop"])
    
    #cleanup team names for each of the leagues
    leagues = ["NHL","MLB","NBA","NFL"]
    for league in leagues:
        cities[league] = cities[league].apply(lambda x:team_name_cleanup(x))
    
    cities = cities[['Metropolitan area','pop',sport.upper()]]
    cities.rename(columns ={sport.upper():"team"}, inplace = True)
    cities = cities[cities["team"]!=""]   
    cities.set_index("team", inplace = True)
    cities.sort_index(inplace = True)

    return cities 

In [67]:
def team_name_cleanup(team):
    
    ##in the team name, replacte +*- with blank. Replace [notes} with blanks
    team = re.sub("\[.+","",team) 
    team = re.sub("[—*+]","",team)
    team = re.sub("\(.+","",team) 
    
    ##I am removing spaces from specific city and team names I have observed in the files. (Note: this approach will not work for a 
    ##very large file with many combinations). Next, I observed, in all files, team names follow the format <city> <team name>. So I
    ##am returing teamname[-1].
    city_names = {"New York":"NewYork",
                  "New Jersey":"NewJersey",
                  "St. Louis":"St.Louis",
                  "Los Angeles":"LosAngeles", 
                  "Tampa Bay":"TampaBay",
                  "Kansas City":"KansasCity",
                  "San Francisco":"SanFrancisco",
                  "San Diego":"SanDiego",
                  "Red Sox":"RedSox",
                  "White Sox":"WhiteSox",
                  "Blue Jackets":"BlueJackets",
                  "Red Wings":"RedWings",
                  "Maple Leafs": "MapleLeafs",
                  "Golden Knights": "GoldenKnights",
                  "Blue Jays": "BlueJays",
                  "New England":"NewEngland",
                  "Green Bay":"GreenBay",
                  "New Orleans":"NewOrleans",
                  "Trail Blazers": "TrailBlazers"}
    
    for name in city_names.keys():
        if team.find(name)!=-1:    #If the team name passed to the function is part of the above dict
            team = team.replace(name,city_names[name])
    
    team = team.strip()            #remove random blanks and such
    if team =="":                  #this would be for cases such as "-" or "[note...]" which become blanks. We will blanks later
        return team
    else: return team.split()[-1]  #for non blanks, return just the team name

In [68]:
"""Calculate the win% correlation (pearson) with the population based on 2018 data. Win% =  wins / (wins + losses)"""

def nhl_calc():

    #loading and cleaning the nhl dataset
    nhl_df = pd.read_csv("datafiles/nhl.csv", usecols=["team","W","L","year","League"])
    nhl_df = nhl_df[(nhl_df["year"]==2018) & (nhl_df["W"].str.isnumeric())] #only 2018 data #removes all rows with non numeric values

    nhl_df["team"] = nhl_df["team"].apply(lambda x:team_name_cleanup(x))

    nhl_df.set_index("team", inplace = True)
    nhl_df.sort_index(inplace = True)

    #merging NY teams, merging LA teams
    win = pd.to_numeric(nhl_df.loc["Rangers","W"]) + pd.to_numeric(nhl_df.loc["Islanders","W"]) + pd.to_numeric(nhl_df.loc["Devils","W"])
    loss = pd.to_numeric(nhl_df.loc["Rangers","L"]) + pd.to_numeric(nhl_df.loc["Islanders","L"]) + pd.to_numeric(nhl_df.loc["Devils","L"])
    nhl_df.loc["RangersIslandersDevils"] = [win, loss, 2018, "NHL"]

    win = pd.to_numeric(nhl_df.loc["Kings","W"]) + pd.to_numeric(nhl_df.loc["Ducks","W"])
    loss = pd.to_numeric(nhl_df.loc["Kings","L"]) + pd.to_numeric(nhl_df.loc["Ducks","L"])
    nhl_df.loc["KingsDucks"] = [win, loss, 2018, "NHL"]
    
    nhl_df.drop(["Rangers","Islanders","Devils","Kings","Ducks"], inplace=True)

    #w/L 
    nhl_df["%W"] = (pd.to_numeric(nhl_df["W"]) / (pd.to_numeric(nhl_df["W"]) + pd.to_numeric(nhl_df["L"])) *100).round(2)

    return nhl_df



def nba_calc():

    #loading and cleaning the nba dataset
    nba_df=pd.read_csv("datafiles/nba.csv", encoding='utf-8', usecols=["team","W","L","year","League"])
    nba_df = nba_df[(nba_df["year"]==2018) & (nba_df["W"].str.isnumeric())] #only 2018 data #removes all rows with non numeric values
    
    nba_df["team"] = nba_df["team"].apply(lambda x:team_name_cleanup(x))
    
    nba_df.set_index("team", inplace = True)
    nba_df.sort_index(inplace = True)


    #merging 1.Lakers/Clippers teams and 2.Knicks/Nets teams
    win = pd.to_numeric(nba_df.loc["Lakers","W"]) + pd.to_numeric(nba_df.loc["Clippers","W"])
    loss = pd.to_numeric(nba_df.loc["Lakers","L"]) + pd.to_numeric(nba_df.loc["Clippers","L"])
    nba_df.loc["LakersClippers"] = [win, loss,2018,"NBA"]

    win = pd.to_numeric(nba_df.loc["Knicks","W"]) + pd.to_numeric(nba_df.loc["Nets","W"])
    loss = pd.to_numeric(nba_df.loc["Knicks","L"]) + pd.to_numeric(nba_df.loc["Nets","L"])
    nba_df.loc["KnicksNets"] = [win, loss,2018,"NBA"]

    nba_df.drop(["Lakers","Clippers","Knicks","Nets"], inplace=True)

    #w/L 
    nba_df["%W"] = (pd.to_numeric(nba_df["W"]) / (pd.to_numeric(nba_df["W"]) + pd.to_numeric(nba_df["L"]))*100).round(2)

    return nba_df


def mlb_calc():

    #loading and cleaning the mlb dataset
    mlb_df=pd.read_csv("datafiles/mlb.csv", usecols = ["team","W","L","year","League"])      
    mlb_df = mlb_df[(mlb_df["year"]==2018)] #only 2018 data #removes all rows with non numeric values
    
    #cleaning up city names
    mlb_df["team"] = mlb_df["team"].apply(lambda x:team_name_cleanup(x))

    mlb_df.set_index("team", inplace = True)
    mlb_df.sort_index(inplace = True)

    #merging 1.Cubs/White Sox teams, 2. Dodgers/Angels teams, 3.Giants/Athletics teams, 4.Yankees/Mets teams
    win = pd.to_numeric(mlb_df.loc["Cubs","W"]) + pd.to_numeric(mlb_df.loc["WhiteSox","W"])
    loss = pd.to_numeric(mlb_df.loc["Cubs","L"]) + pd.to_numeric(mlb_df.loc["WhiteSox","L"])
    mlb_df.loc["CubsWhiteSox"] = [win, loss, 2018, "MLB"]

    win = pd.to_numeric(mlb_df.loc["Dodgers","W"]) + pd.to_numeric(mlb_df.loc["Angels","W"])
    loss = pd.to_numeric(mlb_df.loc["Dodgers","L"]) + pd.to_numeric(mlb_df.loc["Angels","L"])
    mlb_df.loc["DodgersAngels"] = [win, loss, 2018, "MLB"]

    win = pd.to_numeric(mlb_df.loc["Giants","W"]) + pd.to_numeric(mlb_df.loc["Athletics","W"])
    loss = pd.to_numeric(mlb_df.loc["Giants","L"]) + pd.to_numeric(mlb_df.loc["Athletics","L"])
    mlb_df.loc["GiantsAthletics"] = [win, loss, 2018, "MLB"]

    win = pd.to_numeric(mlb_df.loc["Yankees","W"]) + pd.to_numeric(mlb_df.loc["Mets","W"])
    loss = pd.to_numeric(mlb_df.loc["Yankees","L"]) + pd.to_numeric(mlb_df.loc["Mets","L"])
    mlb_df.loc["YankeesMets"] = [win, loss, 2018, "MLB"]

    mlb_df.drop(["Cubs","WhiteSox","Dodgers","Angels","Yankees","Mets","Giants",
                 "Athletics"], inplace=True)

    #w/L 
    mlb_df["%W"] = (pd.to_numeric(mlb_df["W"]) / (pd.to_numeric(mlb_df["W"]) + pd.to_numeric(mlb_df["L"]))*100).round(2)

    return mlb_df



def nfl_calc():
    import pandas as pd
    import re

    #loading and cleaning the nfl dataset
    nfl_df = pd.read_csv("datafiles/nfl.csv", usecols = ["team","W","L","year","League"])      
    nfl_df = nfl_df[(nfl_df["year"]==2018) & (nfl_df["W"].str.isnumeric())] #only 2018 data #removes all rows with non numeric values
    

    #cleaning up city names
    nfl_df["team"] = nfl_df["team"].apply(lambda x:team_name_cleanup(x))

    nfl_df.set_index("team", inplace = True)
    nfl_df.sort_index(inplace = True)
    nfl_df = nfl_df.reindex(["W","L","year","League"], axis = 1)


    #merging 1. 49ers/Raiders teams, 2.Giants/Jets teams 3. Rams/Chargers teams
    win = pd.to_numeric(nfl_df.loc["49ers","W"]) + pd.to_numeric(nfl_df.loc["Raiders","W"])
    loss = pd.to_numeric(nfl_df.loc["49ers","L"]) + pd.to_numeric(nfl_df.loc["Raiders","L"])
    nfl_df.loc["49ersRaiders"] = [win, loss, 2018, "NFL"]
    
    win = pd.to_numeric(nfl_df.loc["Giants","W"]) + pd.to_numeric(nfl_df.loc["Jets","W"])
    loss = pd.to_numeric(nfl_df.loc["Giants","L"]) + pd.to_numeric(nfl_df.loc["Jets","L"])
    nfl_df.loc["GiantsJets"] = [win, loss, 2018, "NFL"]

    win = pd.to_numeric(nfl_df.loc["Rams","W"]) + pd.to_numeric(nfl_df.loc["Chargers","W"])
    loss = pd.to_numeric(nfl_df.loc["Rams","L"]) + pd.to_numeric(nfl_df.loc["Chargers","L"])
    nfl_df.loc["RamsChargers"] = [win, loss, 2018, "NFL"]

    nfl_df.drop(["49ers", "Raiders","Giants", "Jets","Rams","Chargers"], inplace=True)

    
    #w/L 
    nfl_df["%W"] = (pd.to_numeric(nfl_df["W"]) / (pd.to_numeric(nfl_df["W"]) + pd.to_numeric(nfl_df["L"]))*100).round(2)

    return nfl_df

In [69]:
def merge_corr(sport):                                        
    
    if sport == 'nhl':
        df = nhl_calc()
    elif sport == 'nba':    
        df = nba_calc()
    elif sport == 'mlb':  
        df = mlb_calc()
    else:
        df = nfl_calc()
    
    cities = cities_master(sport)
    merge_df = pd.merge(cities, df, on="team")
    corr = stats.pearsonr(merge_df["pop"], merge_df["%W"])[0]

    return merge_df,corr
##-----------------------------------------------------------------------------------------------

In [71]:
def sports_team_performance():

    nhl_df = merge_corr('nhl')[0]
    nhl_df.reset_index(inplace = True)
    nhl_df.set_index("Metropolitan area", inplace = True)
    nhl_df.sort_index(inplace = True)
    
    mlb_df = merge_corr('mlb')[0]
    mlb_df.reset_index(inplace = True)
    mlb_df.set_index("Metropolitan area", inplace = True)
    mlb_df.sort_index(inplace = True)
    
    nba_df=merge_corr('nba')[0]
    nba_df.reset_index(inplace = True)
    nba_df.set_index("Metropolitan area", inplace = True)
    nba_df.sort_index(inplace = True)

    nfl_df=merge_corr('nfl')[0]
    nfl_df.reset_index(inplace = True)
    nfl_df.set_index("Metropolitan area", inplace = True)
    nfl_df.sort_index(inplace = True)

    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    ##NHL NBA
    nhl_nba = pd.merge(nhl_df,nba_df, how ="inner", on= "Metropolitan area")
    p_values.loc["NHL","NBA"] = stats.ttest_rel(nhl_nba["%W_x"], nhl_nba["%W_y"], axis=0)[1]
    p_values.loc["NBA","NHL"] = p_values.loc["NHL","NBA"]
    
    ##NHL MLB
    nhl_mlb = pd.merge(nhl_df,mlb_df, how ="inner", on= "Metropolitan area")
    p_values.loc["NHL","MLB"] = stats.ttest_rel(nhl_mlb["%W_x"], nhl_mlb["%W_y"], axis=0)[1]
    p_values.loc["MLB","NHL"] = p_values.loc["NHL","MLB"]     
    
    ##NHL NFL
    nhl_nfl = pd.merge(nhl_df,nfl_df, how ="inner", on= "Metropolitan area")
    p_values.loc["NHL","NFL"] = stats.ttest_rel(nhl_nfl["%W_x"], nhl_nfl["%W_y"], axis=0)[1]
    p_values.loc["NFL","NHL"] = p_values.loc["NHL","NFL"] 
    
    ##NBA MLB
    nba_mlb = pd.merge(nba_df, mlb_df, how ="inner", on= "Metropolitan area")
    p_values.loc["NBA","MLB"] = stats.ttest_rel(nba_mlb["%W_x"], nba_mlb["%W_y"], axis=0)[1]
    p_values.loc["MLB","NBA"] = p_values.loc["NBA","MLB"] 
    
    ##NBA NFL
    nba_nfl = pd.merge(nba_df, nfl_df, how ="inner", on= "Metropolitan area")
    p_values.loc["NBA","NFL"] = stats.ttest_rel(nba_nfl["%W_x"], nba_nfl["%W_y"], axis=0)[1]
    p_values.loc["NFL","NBA"] = p_values.loc["NBA","NFL"]
    
    ##MLB NFL
    mlb_nfl = pd.merge(mlb_df, nfl_df, how ="inner", on= "Metropolitan area")
    p_values.loc["MLB","NFL"] = stats.ttest_rel(mlb_nfl["%W_x"], mlb_nfl["%W_y"], axis=0)[1]
    p_values.loc["NFL","MLB"] = p_values.loc["MLB","NFL"]
    
    print(p_values)
    return p_values
##-----------------------------------------------------------------------------------

sports_team_performance()

          NFL       NBA       NHL       MLB
NFL       NaN  0.942026  0.030957  0.802627
NBA  0.942026       NaN  0.022328  0.951122
NHL  0.030957  0.022328       NaN  0.000714
MLB  0.802627  0.951122  0.000714       NaN


,NFL,NBA,NHL,MLB
NFL,NaN,0.942026,0.030957,0.802627
NBA,0.942026,NaN,0.022328,0.951122
NHL,0.030957,0.022328,NaN,0.000714
MLB,0.802627,0.951122,0.000714,NaN
